In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 有效广告及其特征

In [2]:
import pandas as pd
from pathlib import Path
import os, re
from functools import reduce, partial
from utils import or_func, add_datepart, split_vals

In [3]:
root = Path('../data')
root
# os.listdir(root)

PosixPath('../data')

In [8]:
users = pd.read_csv(root/'users_data.csv', encoding='utf-8')
users.head()
users.info()

,0,1,2,3,4,5,6,7,8,9,10
0,1,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,54
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",161
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,28
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",12


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38596 entries, 0 to 38595
Data columns (total 11 columns):
0     38596 non-null int64
1     38596 non-null int64
2     38596 non-null int64
3     38596 non-null int64
4     38596 non-null int64
5     38596 non-null int64
6     38596 non-null int64
7     38596 non-null int64
8     38596 non-null object
9     38596 non-null object
10    38596 non-null int64
dtypes: int64(9), object(2)
memory usage: 3.2+ MB


In [18]:
test = pd.read_csv(root/'Btest_sample_new.dat',  sep='\t', header=None, low_memory=False,encoding='utf-8')

test_columns=['样本id','广告id','创建时间','素材尺寸','广告行业id','商品类型','商品id','广告账户id','投放时段','人群定向','出价']
print(len(test_columns))
test.columns=test_columns
test.head()
test.info()

11


,样本id,广告id,创建时间,素材尺寸,广告行业id,商品类型,商品id,广告账户id,投放时段,人群定向,出价
0,1,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,54
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",161
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,28
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",12


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38596 entries, 0 to 38595
Data columns (total 11 columns):
样本id      38596 non-null int64
广告id      38596 non-null int64
创建时间      38596 non-null int64
素材尺寸      38596 non-null int64
广告行业id    38596 non-null int64
商品类型      38596 non-null int64
商品id      38596 non-null int64
广告账户id    38596 non-null int64
投放时段      38596 non-null object
人群定向      38596 non-null object
出价        38596 non-null int64
dtypes: int64(9), object(2)
memory usage: 3.2+ MB


### 多值特征的处理
- 离散为 bool 特征

In [19]:

test.shape

(38596, 11)

## 测试数据

In [20]:
cols = ['age','area','behavior','connectionType','consuptionAbility','device','education','gender','status','work']
test['is_all_field'] = test['人群定向'] == 'all'
test['time'] = test['投放时段']
test.drop(columns='投放时段', inplace=True)
test['创建时间'] = pd.to_datetime(test['创建时间'], unit='s')
test['has_product_id'] = test['商品id'] != -1  # - 1 表示的无商品 id
test = test.reindex(columns=test.columns.to_list() + cols)

### 人群定向的分解

In [21]:
func = partial(split_vals, cols=cols, field='人群定向')

In [22]:
test = test.apply(func, axis=1)
test.drop(columns='人群定向', inplace=True)

- -999 表示空值

In [23]:
test.fillna(-999, inplace=True)

In [24]:
test.head().T

,0,1,2,3,4
样本id,1,2,3,4,5
广告id,152978,507977,639943,605859,742582
创建时间,2016-11-29 02:39:19,2019-03-05 13:52:08,2018-06-21 07:22:37,2019-03-21 09:52:14,2019-02-28 04:52:00
素材尺寸,30,30,34,40,40
广告行业id,118,122,84,98,122
商品类型,3,13,13,13,13
商品id,-1,629,2525,3808,7017
广告账户id,23511,4211,15582,10907,15855
出价,37,54,161,28,12
is_all_field,False,True,False,True,False


- 时间特征

In [25]:
add_datepart(test, '创建时间', drop=True, time=True)

In [26]:
test.head().T

,0,1,2,3,4
样本id,1,2,3,4,5
广告id,152978,507977,639943,605859,742582
素材尺寸,30,30,34,40,40
广告行业id,118,122,84,98,122
商品类型,3,13,13,13,13
商品id,-1,629,2525,3808,7017
广告账户id,23511,4211,15582,10907,15855
出价,37,54,161,28,12
is_all_field,False,True,False,True,False
time,"281474976710655,281474976710655,28147497671065...","70368744161280,70368744161280,70368744161280,7...","281474976694272,281474976694272,28147497669427...","1099243192320,1099243192320,1099243192320,1099...","274936627200,274936627200,274936627200,2749366..."


In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38596 entries, 0 to 38595
Data columns (total 37 columns):
样本id                        38596 non-null int64
广告id                        38596 non-null int64
素材尺寸                        38596 non-null int64
广告行业id                      38596 non-null int64
商品类型                        38596 non-null int64
商品id                        38596 non-null int64
广告账户id                      38596 non-null int64
出价                          38596 non-null int64
is_all_field                38596 non-null bool
time                        38596 non-null object
has_product_id              38596 non-null bool
age                         38596 non-null object
area                        38596 non-null object
behavior                    38596 non-null object
connectionType              38596 non-null object
consuptionAbility           38596 non-null object
device                      38596 non-null object
education                   38596 non-null object
gen

In [28]:
test.shape

(38596, 37)

In [29]:
test.columns

Index(['样本id', '广告id', '素材尺寸', '广告行业id', '商品类型', '商品id', '广告账户id', '出价',
       'is_all_field', 'time', 'has_product_id', 'age', 'area', 'behavior',
       'connectionType', 'consuptionAbility', 'device', 'education', 'gender',
       'status', 'work', 'crt_dateYear', 'crt_dateMonth', 'crt_dateWeek',
       'crt_dateDay', 'crt_dateDayofweek', 'crt_dateDayofyear',
       'crt_dateIs_month_end', 'crt_dateIs_month_start',
       'crt_dateIs_quarter_end', 'crt_dateIs_quarter_start',
       'crt_dateIs_year_end', 'crt_dateIs_year_start', 'crt_dateHour',
       'crt_dateMinute', 'crt_dateSecond', 'crt_dateElapsed'],
      dtype='object')

In [30]:
test['price'] = test['出价']
test.drop(columns=['出价'], inplace=True)

In [31]:
test.to_csv('../data/test_set_B.csv', index=None, encoding='utf-8')

- fin 测试集初步处理